In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


connect to drive 

## Importing Packages 

In [2]:
!pip install pyspark # install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 48.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=db40e2a18771679bd07e30caa2d4e2bf85cc1ab1f2d1d4b55c2f9ead00b4783e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
#import libary
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn 

#spark libary
import pyspark
from pyspark.sql import types
from pyspark.sql.functions import col, isnan, when, count, explode, array, lit
from pyspark.sql.types import FloatType #to change type to float
from pyspark.sql.types import IntegerType #to change type to integer
#import spark feature libary
from pyspark.ml.feature import Imputer, VectorAssembler, StringIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#import classfication model libary
# Import some classifiers and multiclass evaluator
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline#pipline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

warnings.filterwarnings('ignore')

## Loading Dataset 

In [4]:
pandadf = pd.read_csv('/content/drive/MyDrive/cleaning_data.csv') #read ssv file as pandas

## connect to  spark 

In [5]:
from pyspark.sql import SparkSession # import SparkSession


In [6]:
#connect spark
spark = pyspark.sql.SparkSession.builder.config("spark.executor.memory", "16g").config("spark.driver.memory", "16g").getOrCreate()

In [7]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


In [8]:
#convert from pandas to spark
df = spark.createDataFrame(pandadf)


In [9]:
#SHOW DATA
df

DataFrame[restaurant_name: string, country: string, region: string, city: string, latitude: double, longitude: double, claimed: string, popularity_detailed: string, popularity_generic: string, cuisines: string, vegetarian_friendly: bigint, vegan_options: bigint, gluten_free: bigint, open_days_per_week: bigint, open_hours_per_week: double, working_shifts_per_week: bigint, avg_rating: double, total_reviews_count: bigint, default_language: string, reviews_count_in_default_language: bigint, excellent: double, very_good: double, average: double, poor: double, terrible: double, food: double, service: double, value: double, atmosphere: double, price_range: string]

In [10]:
print('Shape of the dataset: ',(df.count(), len(df.columns)))# number of rows and coloumn

Shape of the dataset:  (749544, 30)


the dataset contain 30 columns and 749544 rows

## Feature Engineering and Feature Selection

In [ ]:
data.printSchema()#show 

root
 |-- restaurant_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- claimed: string (nullable = true)
 |-- popularity_detailed: string (nullable = true)
 |-- popularity_generic: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- vegetarian_friendly: long (nullable = true)
 |-- vegan_options: long (nullable = true)
 |-- gluten_free: long (nullable = true)
 |-- open_days_per_week: long (nullable = true)
 |-- open_hours_per_week: double (nullable = true)
 |-- working_shifts_per_week: long (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- total_reviews_count: long (nullable = true)
 |-- default_language: string (nullable = true)
 |-- reviews_count_in_default_language: long (nullable = true)
 |-- excellent: double (nullable = true)
 |-- very_good: double (nullable = true)
 |-- 

## split avg_rating to 3 classes

In [12]:



spark = data.replace( [1.0,1.5,2.0,2.5] , 0, subset=["avg_rating"]) #replace avg_rating  values 1-2.5 to 0




In [13]:
spark = spark.replace([3,3.5,4.0] , 1, subset=["avg_rating"]) #replace avg_rating values 3-4 to 1


In [14]:
spark = spark.replace([4.5,5.0] , 2, subset=["avg_rating"]) #replace avg_rating values 4.5 -5 to 2


## convert type for some column

In [16]:
spark = spark.withColumn("vegetarian_friendly", spark["vegetarian_friendly"].cast(IntegerType())) #convert vegetarian_friendly type to  int type


In [17]:
spark = spark.withColumn("avg_rating", spark["avg_rating"].cast(IntegerType())) #convert avg_rating type to  int type


In [18]:
spark = spark.withColumn("vegan_options", spark["vegan_options"].cast(IntegerType())) #convert vegan_options type to  int type


In [19]:
spark = spark.withColumn("reviews_count_in_default_language", spark["reviews_count_in_default_language"].cast(IntegerType())) #convert reviews_count_in_default_language type to  int type


In [ ]:
spark.groupBy("avg_rating").count().show()#check 

## - Baseline --> is 1 --> would be accurate by (408493/749544)*100 = 54%
## - Whenever we build a model it needs to be more accurate than 54% 

+----------+------+
|avg_rating| count|
+----------+------+
|         1|408493|
|         2|313881|
|         0| 27150|
+----------+------+



### convert unbalance data to balance data

In [20]:
major_df = spark.filter(col("avg_rating") == 2)
minor_df = spark.filter(col("avg_rating") == 0)
major2_df = spark.filter(col("avg_rating") == 1)



#we will calculate the ratio to determine the difference between the number of avg_rating 1 and avg_rating 0 transactions.
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

ratio: 11


In [21]:
#create a range on this ratio and store it in variable a
a = range(ratio)

#duplicate the minority rows
oversampled_df = minor_df.withColumn("dummy", explode(array([lit(x) for x in a]))).drop('dummy')

In [22]:
oversampled_df.count()

298650

class 0 oversampling and now 298650

In [23]:
combined = major_df.unionAll(oversampled_df)

combined_df = combined.unionAll(major2_df)


In [ ]:
combined_df.printSchema()

root
 |-- restaurant_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- claimed: string (nullable = true)
 |-- popularity_detailed: string (nullable = true)
 |-- popularity_generic: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- vegetarian_friendly: integer (nullable = true)
 |-- vegan_options: integer (nullable = true)
 |-- gluten_free: long (nullable = true)
 |-- open_days_per_week: long (nullable = true)
 |-- open_hours_per_week: double (nullable = true)
 |-- working_shifts_per_week: long (nullable = true)
 |-- avg_rating: integer (nullable = true)
 |-- total_reviews_count: long (nullable = true)
 |-- default_language: string (nullable = true)
 |-- reviews_count_in_default_language: integer (nullable = true)
 |-- excellent: double (nullable = true)
 |-- very_good: double (nullable = t

In [24]:
#This step will label encode all the categorical columns and store them in different columns with the same name + '_idx', 
#so category will become category_idx 
cat_cols = ['price_range','cuisines','claimed'] #cuisines and price_range

#StringIndexer() is equivalent to LabelEncoder()
for c in cat_cols: 
    indexer = StringIndexer(inputCol=c, outputCol=c+'_idx') #we pass the columns from the list as input one by one
    combined_df = indexer.fit(combined_df).transform(combined_df) #here we fit and transform the data altogether
    
final_df2 = combined_df.drop(*cat_cols) #we will drop all the categorical columns we defined earlier

## Split Data

In [25]:
#drop unneed columns

final_df=final_df2.drop('service','value','average','reviews_count_in_default_language','price_range', 'atmosphere','default_language','working_shifts_per_week','open_hours_per_week','open_days_per_week','popularity_generic','popularity_detailed','longitude','latitude','region','city','country','restaurant_name')

cols = final_df.columns #extract the column names from the dataframe
cols.remove('avg_rating') #remove avg_rating -> we need this to be our label

#vector assembler will take all the columns and convert them into one column called features
assembler = VectorAssembler(inputCols=cols, outputCol='features')

#the .transform will apply the changes here
final_df = assembler.transform(final_df)

In [ ]:
#we can now see that features column will appear within the dataframe
final_df

DataFrame[vegetarian_friendly: int, vegan_options: int, gluten_free: bigint, avg_rating: int, total_reviews_count: bigint, excellent: double, very_good: double, poor: double, terrible: double, food: double, price_range_idx: double, cuisines_idx: double, claimed_idx: double, features: vector]


Lets Split

80% in training set and 20% is testing set.

In [26]:
# We will now create a new dataframe only consisting of the features column and the label column (actually avg_rating column but renamed)
df_data = final_df.select(col('features'), col('avg_rating').alias('label'))

#simple data splitting
df_train, df_test = df_data.randomSplit([0.8, 0.2])

## Train Models



### 1st  Model 

In [29]:
# Decision Tree
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features" ,maxDepth=5,maxBins=800000)
model_dt = dt.fit(df_train)

### 2nd  Model  

In [31]:
# Random Forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=8,maxDepth=5 ,maxBins=750000)
model_rf = rf.fit(df_train)

### 3rd  Model  

In [32]:
# Logistic Regression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol="label", featuresCol="features")
model_lr = lr.fit(df_train)

### 4  Model  

In [33]:
#NaiveBayes
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0, 
                modelType="gaussian", 
                featuresCol='features', labelCol='label')
model_niv = nb.fit(df_train)

## Model Evaluation



### 1st Model  Evaluation

In [34]:
#  DT .transform
pred_dt = model_dt.transform(df_test)


## report  and matrix


In [35]:
#report
y_true = pred_dt.select(['label']).collect()
y_pred = pred_dt.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.90      0.85     59810
           1       0.81      0.65      0.72     82152
           2       0.76      0.86      0.81     62698

    accuracy                           0.79    204660
   macro avg       0.79      0.80      0.79    204660
weighted avg       0.79      0.79      0.78    204660



The accuracy of D tree is .79

In [36]:
#matrix
from sklearn import metrics
matrix=metrics.confusion_matrix(y_true,y_pred)
print(matrix)

[[53642  5211   957]
 [12160 53532 16460]
 [ 1213  7328 54157]]


### 2nd Model  Evaluation

In [37]:
# random forst.transform
pred_rf = model_rf.transform(df_test)


## report  and matrix


In [38]:

y_true3 = pred_rf.select(['label']).collect()
y_pred3 = pred_rf.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true3, y_pred3))

              precision    recall  f1-score   support

           0       0.82      0.92      0.87     59810
           1       0.83      0.68      0.74     82152
           2       0.77      0.86      0.81     62698

    accuracy                           0.80    204660
   macro avg       0.81      0.82      0.81    204660
weighted avg       0.81      0.80      0.80    204660



The accuracy of Random forst is .80

In [39]:
from sklearn import metrics
matrix=metrics.confusion_matrix(y_true3,y_pred3)
print(matrix)

[[55083  3920   807]
 [11450 55495 15207]
 [  843  7727 54128]]


### 3rd Model  Evaluation

In [40]:
# Logistic .transform
pred_lr = model_lr.transform(df_test)


## report  and matrix


In [41]:
y_true4 = pred_lr.select(['label']).collect()
y_pred4 = pred_lr.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true4, y_pred4))

              precision    recall  f1-score   support

           0       1.00      0.00      0.01     59810
           1       0.40      1.00      0.57     82152
           2       0.00      0.00      0.00     62698

    accuracy                           0.40    204660
   macro avg       0.47      0.33      0.19    204660
weighted avg       0.45      0.40      0.23    204660



The accuracy of Logistic is .40

In [42]:
from sklearn import metrics
matrix=metrics.confusion_matrix(y_true4,y_pred4)
print(matrix)

[[  159 59651     0]
 [    0 82152     0]
 [    0 62698     0]]


### 4th Model  Evaluation


In [43]:
#NaiveBayes
pred_nb = model_niv.transform(df_test)

## report  and matrix


In [44]:
y_true6 = pred_nb.select(['label']).collect()
y_pred6 = pred_nb.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true6, y_pred6))

              precision    recall  f1-score   support

           0       0.39      0.96      0.56     59810
           1       0.82      0.15      0.26     82152
           2       0.64      0.44      0.52     62698

    accuracy                           0.48    204660
   macro avg       0.62      0.52      0.45    204660
weighted avg       0.64      0.48      0.43    204660



The accuracy of NaiveBayes is .48

In [45]:
from sklearn import metrics
matrix=metrics.confusion_matrix(y_true6,y_pred6)
print(matrix)

[[57535  1556   719]
 [54502 12719 14931]
 [33682  1189 27827]]


## Show ML Evaluation as Dataframe


In [46]:
# Accuracy Metric
evaluator_A = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# F1 Metric
evaluator_F = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

# Weighted Precision
evaluator_P = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")

# Weighted Recall
evaluator_R = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

# Our models
models = [pred_dt, pred_rf, pred_lr,pred_nb]

# Empty lists that will store the scores for each metric for each model.
accuracy = []
F1 = []
precision = []
recall = []

# Simple loop to populate the empty lists with scores of models for each metric.
for model in models:
    accuracy.append(evaluator_A.evaluate(model))
    F1.append(evaluator_F.evaluate(model))
    precision.append(evaluator_P.evaluate(model))
    recall.append(evaluator_R.evaluate(model))

In [47]:
# We will convert all lists created above into a dataframe for easy viewing.
df_ev = pd.DataFrame(list(zip(accuracy, F1, precision, recall)), 
                     columns = ['Accuracy', 'F1-Score', 'Weighted Precision', 'Weighted Recall'],
                     index = ['Decision Tree', 'Random Forest', 'Logistic Regression','nive'])

In [48]:
df_ev

,Accuracy,F1-Score,Weighted Precision,Weighted Recall
Decision Tree,0.788288,0.784283,0.790955,0.788288
Random Forest,0.804779,0.801209,0.807106,0.804779
Logistic Regression,0.402184,0.231629,0.453494,0.402184
nive,0.479239,0.428802,0.641618,0.479239


the best model is random tree with 0.80 Accuracy , and the worest model is Logistic Regression	 with 0.40 


## select  best model

Random Forst is the best model with accurcy 0.80 and 


# ML Tuning for Best Model

In [49]:

#initialize  grid -> we are using variation with only one parameter called maxIter (maximum iteration)
grid = ParamGridBuilder().addGrid(rf.maxDepth, [5,6]).build()

#CrossValidator 
cv = CrossValidator(estimator=rf, estimatorParamMaps=grid, evaluator=evaluator_A, parallelism=2)

# fit training set
cvModel = cv.fit(df_train)

In [50]:
#average metrics
cvModel.avgMetrics 


[0.8056909650786443, 0.8191410400491006]

In [51]:
# the accuracy of our model on the testing set
evaluator_A.evaluate(cvModel.transform(df_test))

0.8252858399296394

after apply grid the accuracy is 82

# ML Pipeline for Best Model

In [ ]:
pipeline = Pipeline(stages=[rf])


In [ ]:

# Fit the pipeline to training documents.
model = pipeline.fit(df_train)

In [ ]:
#the accuracy of our model on the testing set
evaluator_A.evaluate(model.transform(df_test))

0.8019790846733386

after apply pipeline the accuracy is 80